# Explore here

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno
from sklearn.impute import KNNImputer
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import xgboost as xgb
import requests

In [2]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/data-preprocessing-project-tutorial/main/AB_NYC_2019.csv"
response = requests.get(url)

if response.status_code == 200:
    with open('/workspace/machine-learning/data/raw/downloaded_file.csv', 'wb') as file:
        file.write(response.content)
    print("CSV file downloaded successfully.")

CSV file downloaded successfully.


In [3]:
df = pd.read_csv('/workspace/machine-learning/data/raw/downloaded_file.csv').iloc[:,1:]
pd.set_option('display.max_columns', None)
df.head()

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [4]:
df.tail()

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
48890,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2
48894,Trendy duplex in the very heart of Hell's Kitchen,68119814,Christophe,Manhattan,Hell's Kitchen,40.76404,-73.98933,Private room,90,7,0,NaN,NaN,1,23


In [5]:
df.shape

(48895, 15)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            48879 non-null  object 
 1   host_id                         48895 non-null  int64  
 2   host_name                       48874 non-null  object 
 3   neighbourhood_group             48895 non-null  object 
 4   neighbourhood                   48895 non-null  object 
 5   latitude                        48895 non-null  float64
 6   longitude                       48895 non-null  float64
 7   room_type                       48895 non-null  object 
 8   price                           48895 non-null  int64  
 9   minimum_nights                  48895 non-null  int64  
 10  number_of_reviews               48895 non-null  int64  
 11  last_review                     38843 non-null  object 
 12  reviews_per_month               

In [7]:
# We consider that we don't need the column host_name, name, host_id, last_review, reviews_per_month, neighbourhood_group.
df.drop(columns = ["host_name", "name", "host_id","last_review","reviews_per_month","neighbourhood_group"], inplace = True)

In [8]:
df.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
0,Kensington,40.64749,-73.97237,Private room,149,1,9,6,365
1,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2,355
2,Harlem,40.80902,-73.94190,Private room,150,3,0,1,365
3,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,1,194
4,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,1,0


In [9]:
# We confirm the columns were erased correctly.
print(df.columns)

Index(['neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


In [10]:
# We check if there are duplicate rows.
print(df.duplicated().sum())

0


In [11]:
# We check which columns have a NAN value.
df.isnull().any(axis=0)

neighbourhood                     False
latitude                          False
longitude                         False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

In [12]:
# We create a new table to identify the listings that have a review.
df['has_review'] = (df['number_of_reviews']>0).astype(int)

In [13]:
df.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,has_review
0,Kensington,40.64749,-73.97237,Private room,149,1,9,6,365,1
1,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2,355,1
2,Harlem,40.80902,-73.94190,Private room,150,3,0,1,365,0
3,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,1,194,1
4,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,1,0,1


In [14]:
a = df['room_type'].unique()
a

array(['Private room', 'Entire home/apt', 'Shared room'], dtype=object)

In [15]:
# We will specify the values in the mapping (0 = shared room, 1 = private room, 2 = house).
mapping = {'Shared room': 0, 'Private room': 1, 'Entire home/apt': 2}

df['room_type'] = df['room_type'].map(mapping)

In [16]:
# We check the dataframe.
df.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,has_review
0,Kensington,40.64749,-73.97237,1,149,1,9,6,365,1
1,Midtown,40.75362,-73.98377,2,225,1,45,2,355,1
2,Harlem,40.80902,-73.94190,1,150,3,0,1,365,0
3,Clinton Hill,40.68514,-73.95976,2,89,1,270,1,194,1
4,East Harlem,40.79851,-73.94399,2,80,10,9,1,0,1


In [17]:
# We will now do target encoding for the neighbourhood. 
df['room_type_str'] = df['room_type'].astype(str) #We create a new column to 
# We will take into account the room type for the mean value of the neighbourhood.
df['neighbourhood_room_type'] = df['neighbourhood'] + '_' + df['room_type_str']


In [18]:
df.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,has_review,room_type_str,neighbourhood_room_type
0,Kensington,40.64749,-73.97237,1,149,1,9,6,365,1,1,Kensington_1
1,Midtown,40.75362,-73.98377,2,225,1,45,2,355,1,2,Midtown_2
2,Harlem,40.80902,-73.94190,1,150,3,0,1,365,0,1,Harlem_1
3,Clinton Hill,40.68514,-73.95976,2,89,1,270,1,194,1,2,Clinton Hill_2
4,East Harlem,40.79851,-73.94399,2,80,10,9,1,0,1,2,East Harlem_2


In [19]:
# We now calculate the average price for each neighborhood and room type combination
avg_prices = df.groupby('neighbourhood_room_type')['price'].mean()

In [20]:
# We now add the average prices back to the original dataframe
df['neighbourhood_encoding'] = df['neighbourhood_room_type'].map(avg_prices)

In [21]:
df.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,has_review,room_type_str,neighbourhood_room_type,neighbourhood_encoding
0,Kensington,40.64749,-73.97237,1,149,1,9,6,365,1,1,Kensington_1,59.377551
1,Midtown,40.75362,-73.98377,2,225,1,45,2,355,1,2,Midtown_2,301.268182
2,Harlem,40.80902,-73.94190,1,150,3,0,1,365,0,1,Harlem_1,81.950413
3,Clinton Hill,40.68514,-73.95976,2,89,1,270,1,194,1,2,Clinton Hill_2,243.982659
4,East Harlem,40.79851,-73.94399,2,80,10,9,1,0,1,2,East Harlem_2,189.545267


In [28]:
# We will compare the kensington neighborhood
df.loc[df['neighbourhood_room_type'] == 'Midtown_1']

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,has_review,room_type_str,neighbourhood_room_type,neighbourhood_encoding
618,Midtown,40.75575,-73.96842,1,110,2,61,1,264,1,1,Midtown_1,244.42891
943,Midtown,40.74553,-73.98943,1,135,3,214,2,167,1,1,Midtown_1,244.42891
997,Midtown,40.74530,-73.99056,1,145,3,165,2,181,1,1,Midtown_1,244.42891
1439,Midtown,40.74648,-73.98459,1,100,3,41,1,0,1,1,Midtown_1,244.42891
1539,Midtown,40.76221,-73.97787,1,120,4,31,1,0,1,1,Midtown_1,244.42891
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47967,Midtown,40.76384,-73.98248,1,125,1,6,1,142,1,1,Midtown_1,244.42891
48000,Midtown,40.75258,-73.97111,1,134,30,0,12,365,0,1,Midtown_1,244.42891
48023,Midtown,40.75669,-73.96848,1,100,2,0,1,89,0,1,Midtown_1,244.42891
48576,Midtown,40.75286,-73.99297,1,120,2,1,1,7,1,1,Midtown_1,244.42891


In [29]:
# WE DROP THE COLUMNS THAT ARE NOT NECESSARY ANYMORE.
df.drop(columns = ["neighbourhood", "room_type_str", "neighbourhood_room_type"], inplace = True)
df.head()

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,has_review,neighbourhood_encoding
0,40.64749,-73.97237,1,149,1,9,6,365,1,59.377551
1,40.75362,-73.98377,2,225,1,45,2,355,1,301.268182
2,40.80902,-73.94190,1,150,3,0,1,365,0,81.950413
3,40.68514,-73.95976,2,89,1,270,1,194,1,243.982659
4,40.79851,-73.94399,2,80,10,9,1,0,1,189.545267
